In [1]:
import sys
import pandas as pd
import numpy as np
import time
sys.path.append('../script/')
sys.path.append('../class/')
import sunnytools as st
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#LOG IN INTO SIAMCHART
import siamchartdatareader as scdr
dr=scdr.SiamChartDataReader()
dr.loginToSiamChart()

all_thai_tickers = st.getAllThaiTickers()

In [3]:
##### FOR Getting Tickers Not Yet Loaded ######
import os
all_thai_tickers = st.getAllThaiTickers()
tickers_success = set()
directory = os.fsencode("/Users/Sunny/GoogleDrive/SunnyDoc/Github/project-potzy/data/th/fs/")
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        tickers_success.add(filename[:-4])
tickers_fail = set(all_thai_tickers).difference(tickers_success)
print(len(tickers_fail))

1


In [ ]:
#initialize (modify here for rerun)

#tickers_left=set(all_thai_tickers)
tickers_left=tickers_fail

df_div_all=pd.DataFrame()
df_fs_all=pd.DataFrame()
bad_tickers=set()
tickers_loaded=set()

print(len(tickers_left))
tickers_left

In [ ]:
num_round=0
while len(tickers_left) > 0 and num_round < 10:
    for ticker in tickers_left:
        try:
            tryRelogin=False
            while True:
                print('Loading data for ',ticker)
                df_fs=dr.getFinancialData(ticker)
                df_div=dr.getDividendData(ticker)
                print('Done for ticker ',ticker,' size of dividend and fs df are ',len(df_div),len(df_fs))
                if len(df_fs) <= 5 and not tryRelogin:
                    try:
                        dr.loginToSiamChart()
                    except:
                        print('log in not successful, probably stil in login session')
                    tryRelogin=True
                    time.sleep(1)
                    print("relogin and reload the data")
                else:
                    break

            #reorganize columns
            prefix_cols=['revenue','net_profit','eps','dps','gross_profit','ebit','ebitda']
            cols=prefix_cols+[x for x in df_fs.columns if x not in prefix_cols]
            df_fs=df_fs[cols]

            #save result
            df_fs.to_csv('../data/th/fs/'+ticker+'.csv')
            df_div.to_csv('../data/th/div/'+ticker+'.csv')
            
            #
            tickers_loaded.add(ticker)

        except:
            #raise
            print('Error loading information for ticker ',ticker)
            bad_tickers.add(ticker)
            
    tickers_left = tickers_left.difference(tickers_loaded)
    num_round+=1
    

In [ ]:
# #### OLD CODE : SINGLE FILE ####

# for ticker in ticker_list:
#     try:
#         tryRelogin=False
#         while True:
#             print('Loading data for ',ticker)
#             df_fs=dr.getFinancialData(ticker)
#             df_div=dr.getDividendData(ticker)
#             print('Done for ticker ',ticker,' size of dividend and fs df are ',len(df_div),len(df_fs))
#             if len(df_fs) <= 5 and not tryRelogin:
#                 try:
#                     dr.loginToSiamChart()
#                 except:
#                     print('log in not successful, probably stil in login session')
#                 tryRelogin=True
#                 time.sleep(1)
#                 print("relogin and reload the data")
#             else:
#                 break

#         #add ticker
#         df_div.insert(0,'ticker',ticker)
#         df_fs.insert(0,'ticker',ticker)

#         #append to the big dataframe
#         df_div_all=df_div_all.append(df_div)
#         df_fs_all=df_fs_all.append(df_fs)
    
#     except:
#         #raise
#         print('Error loading information for ticker ',ticker)
#         bad_ticker.append(ticker)

# #reorganize columns
# prefix_cols=['ticker','revenue','net_profit','eps','dps','gross_profit','ebit','ebitda']
# cols=prefix_cols+[x for x in df_fs_all.columns if x not in prefix_cols]
# df_fs_all=df_fs_all[cols]